PyTorch Transformer from Scratch 

In [1]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
  def __init__(self, embed_size, heads):
    # initialize parent class
    super(SelfAttention, self).__init__()
    self.embed_size = embed_size
    self.heads = heads
    self.head_dim = embed_size // heads

    # check that 256 embedding size can't be split into 7 parts, or any other number that is not possible
    assert (self.head_dim * heads == embed_size), "Incorrect embedding size. Needs to be divisible by heads"
    
    # linear layer that we send out VKQ through
    # defining VKQ below
    self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.fc_out = nn.Linear(heads*self.head_dim, embed_size)

  def forward(self, values, keys, query, mask):
    # define number of training examples
    N = query.shape[0]
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

    # split embedding into self.heads pieces
    values = values.reshape(N, value_len, self.heads, self.head_dim) 
    keys = keys.reshape(N, key_len, self.heads, self.head_dim)
    queries = query.reshape(N, key_len, self.heads, self.head_dim)

    # matrix multiplication of q and V
    # einstein summation notation
    energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

    if mask is not None:
      energy = energy.masked_fill(mask == 0, float("-1e20"))

    # running this through the softmax() in the self attention function
    attention = torch.softmax(energy / (self.embed_size**(1/2)), dim=3)

    # output: N, query_len, heads, head_dim) 
    out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(N, query_len, self.heads*self.head_dim)

    out = self.fc_out(out)
    return out

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TransformerBlock(nn.module):
  def __init__(self, embed_size, heads, dropout, forward_expansion):
    super(TransformerBlock, self).__init__()
    self.attention = SelfAttention(embed_size, heads)

    # first normalization layer
    # layer norm takes an average for every example
    self.norm1 = nn.LayerNorm(embed_size)

    self.norm2 = nn.LayerNorm(embed_size)

    # feed forward
    self.feed_forward = nn.Sequential(
      # in, out
      nn.Linear(embed_size, forward_expansion * embed_size),
      nn.ReLU(),
      nn.Linear(forward_expansion*embed_size, embed_size)
      
    )

    self.dropout = nn.Dropout(dropout)

  def forward(self, value, key, query, mask):
    attention = self.attention(value, key, query, mask)
    #skip connection
    self.norm1(attention + query)
    X = self.dropout(self.norm1(attention+query))
    forward = self.feed_forward(X)
    out = self.dropout(self.norm2(forward + X))
    return out


AttributeError: module 'torch.nn' has no attribute 'module'

In [3]:
class Encoder(nn.Module):
  def __init__(self, src_vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
    super(Encoder, self).__init__()

    self.embed_size = embed_size
    self.device = device
    self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
    self.positional_embedding = nn.Embedding(max_length, embed_size)

    self.layers = nn.ModuleList(
      [TransformerBlock(embed_size, heads, dropout=dropout, forward_expansion=forward_expansion)]
    )
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask):
    N, seq_length = x.shape
    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

    out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

    for layer in self.layers:
      out = layer(out, out, out, mask)

    return out

In [4]:
class DecoderBlock(nn.Module):
  def __init__(self, embed_size, heads, forward_expansion, dropout, device):
    super(DecoderBlock,self).__init__()

    # send it through an attention layer
    self.attention = SelfAttention(embed_size,heads)
    self.norm = nn.LayerNorm(embed_size)

    self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)

    self.dropout = nn.Dropout(dropout)
  
  def forward(self, x, value, key, src_mask, trg_mask):
    attention = self.attention(x, x, x, trg_mask)
    query = self.dropout(self.norm(attention + x))

    out = self.transformer_block(value, key, query, src_mask)
    return out

In [ ]:
class Decoder(nn.Module):
  def __init__(self, trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, device, max_length):
    super(Decoder, self).__init__()


    self.device = device
    self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
    self.position_embedding = nn.Embedding(max_length, embed_size)

    self.layers = nn.ModuleList(
      [DecoderBlock(embed_size, heads, forward_expansion, dropout, device) for _ in range(num_layers)]
    )

    self.fc_out = nn.Linear(embed_size, trg_vocab_size)

    self.dropout = nn.Dropout(dropout)

  def forward(self, x, enc_out, src_mask, trg_mask):
    N, seq_length = x.shape

    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

    x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

    for layer in self.layers:
      x = layer(x, enc_out, enc_out, src_mask, trg_mask)

    out = self.fc_out(x)
